In [1]:
import numpy as np
import pandas as pd
import csv
import ast
import collections
import matplotlib.pyplot as plt
import Levenshtein as lev
from fuzzywuzzy import fuzz 
import datetime
import matplotlib.pyplot as plt
import re
from urllib.parse import urlparse
from collections import Counter

from elsapy.elsclient import ElsClient
from elsapy.elsdoc import FullDoc, AbsDoc
from elsapy.elssearch import ElsSearch

import time # for sleep
from pybtex.database import parse_file, BibliographyData, Entry
import json
from elsapy.elsclient import ElsClient
from elsapy.elsdoc import AbsDoc
from elsapy.elssearch import ElsSearch

In [2]:
def AffiliationsFromScopusByDOI( client, doi ):
    doc_srch = ElsSearch("DOI("+doi+")",'scopus')
    doc_srch.execute(client, get_all = True)
    # print ("doc_srch has", len(doc_srch.results), "results.")
    # print(doc_srch.results)
    try:
        scopus_id=doc_srch.results[0]["dc:identifier"].split(":")[1]
        scp_doc = AbsDoc(scp_id = scopus_id)
        if scp_doc.read(client):
            # print ("scp_doc.title: ", scp_doc.title)
            scp_doc.write()   
        else:
            print ("Read document failed.")
        # print(scp_doc.data["affiliation"])
        return scp_doc.data
    except:
        return None

In [3]:
## Load configuration
con_file = open("config.json")
config = json.load(con_file)
con_file.close()
## Initialize client
client = ElsClient(config['apikey'])
# https://api.elsevier.com/content/search/scopus?query=DOI(10.1109/MCOM.2016.7509373)&apiKey=6d485ef1fe1408712f37e8a783a285a4

In [4]:
df_current_extra_info = pd.read_pickle('extra_info_CS5099.pkl')
df_current_extra_info

,affiliation,coredata
0,"[{'affiliation-city': None, 'affilname': 'Pata...","{'srctype': 'j', 'prism:issueIdentifier': '230..."
1,"[{'affiliation-city': 'New Delhi', 'affilname'...","{'srctype': 'j', 'eid': '2-s2.0-85083171050', ..."
2,None,None
3,"[{'affiliation-city': 'London', 'affilname': '...","{'srctype': 'j', 'eid': '2-s2.0-79953057246', ..."
4,"[{'affiliation-city': 'Baoding', 'affilname': ...","{'srctype': 'j', 'prism:issueIdentifier': '7',..."
...,...,...
11659,None,None
11660,"{'affiliation-city': 'Regensburg', 'affilname'...","{'srctype': 'j', 'eid': '2-s2.0-85097251977', ..."
11661,"{'affiliation-city': 'Safat', 'affilname': 'Al...","{'srctype': 'j', 'prism:issueIdentifier': '1',..."
11662,"[{'affiliation-city': 'Atlanta', 'affilname': ...","{'srctype': 'j', 'prism:issueIdentifier': '1',..."


In [5]:
dict_extra_info = dict()
dict_extra_info[0] = AffiliationsFromScopusByDOI(client, '10.1109/MCOM.2016.7509373')
dict_extra_info[1] = AffiliationsFromScopusByDOI(client, '10.1109/MCOM.2016.7509373')

In [6]:
df_current_extra_info = pd.read_pickle('extra_info_CS5099.pkl')
df_current_extra_info

,affiliation,coredata
0,"[{'affiliation-city': None, 'affilname': 'Pata...","{'srctype': 'j', 'prism:issueIdentifier': '230..."
1,"[{'affiliation-city': 'New Delhi', 'affilname'...","{'srctype': 'j', 'eid': '2-s2.0-85083171050', ..."
2,None,None
3,"[{'affiliation-city': 'London', 'affilname': '...","{'srctype': 'j', 'eid': '2-s2.0-79953057246', ..."
4,"[{'affiliation-city': 'Baoding', 'affilname': ...","{'srctype': 'j', 'prism:issueIdentifier': '7',..."
...,...,...
11659,None,None
11660,"{'affiliation-city': 'Regensburg', 'affilname'...","{'srctype': 'j', 'eid': '2-s2.0-85097251977', ..."
11661,"{'affiliation-city': 'Safat', 'affilname': 'Al...","{'srctype': 'j', 'prism:issueIdentifier': '1',..."
11662,"[{'affiliation-city': 'Atlanta', 'affilname': ...","{'srctype': 'j', 'prism:issueIdentifier': '1',..."


In [7]:
%%time
CORD19_CSV = pd.read_csv('../data/cord-19/CORD19_software_mentions.csv')
doi = CORD19_CSV['doi']
doi
#delete Nans
doi_counted = doi.value_counts()
doi_counted
i = 0
dict_new_extra_info = dict()

while i < 10:
    print("Position: " + str(i) + " -> " +  doi_counted.index[i])
    dict_new_extra_info[i] = AffiliationsFromScopusByDOI(client, doi_counted.index[i])
    i = i + 1 

Position: 0 -> 10.31729/jnma.5498
Position: 1 -> 10.1016/j.dsx.2020.04.012
Position: 2 -> 10.1371/journal.pone.0000679
Position: 3 -> 10.1093/ijpp/riaa003
Position: 4 -> 10.1007/s10880-020-09724-6
Position: 5 -> 10.1016/j.energy.2014.02.062


KeyboardInterrupt: 

In [8]:
dict_new_extra_info

{0: {'affiliation': [{'affiliation-city': None,
    'affilname': 'Patan Academy of Health Sciences-School of Medicine',
    'affiliation-country': 'Nepal'},
   {'affiliation-city': 'Valhalla',
    'affilname': 'New York Medical College',
    'affiliation-country': 'United States'},
   {'affiliation-city': 'New Delhi',
    'affilname': 'All India Institute of Medical Sciences, New Delhi',
    'affiliation-country': 'India'}],
  'coredata': {'srctype': 'j',
   'prism:issueIdentifier': '230',
   'eid': '2-s2.0-85092733043',
   'prism:coverDate': '2020-10-01',
   'prism:aggregationType': 'Journal',
   'prism:url': 'https://api.elsevier.com/content/abstract/scopus_id/85092733043',
   'subtypeDescription': 'Article',
   'dc:creator': {'author': [{'ce:given-name': 'Carmina',
      'preferred-name': {'ce:given-name': 'Carmina',
       'ce:initials': 'C.',
       'ce:surname': 'Shrestha',
       'ce:indexed-name': 'Shrestha C.'},
      '@seq': '1',
      'ce:initials': 'C.',
      '@_fa': 'true

In [9]:
entry = AffiliationsFromScopusByDOI(client, '10.1109/MCOM.2016.7509373')

In [10]:
print (json.dumps(entry, ensure_ascii=False, indent=4))

{
    "affiliation": [
        {
            "affiliation-city": "Lahore",
            "affilname": "Information Technology University",
            "affiliation-country": "Pakistan"
        },
        {
            "affiliation-city": "Trieste",
            "affilname": "Abdus Salam International Centre for Theoretical Physics",
            "affiliation-country": "Italy"
        },
        {
            "affiliation-city": "Cambridge",
            "affilname": "University of Cambridge",
            "affiliation-country": "United Kingdom"
        },
        {
            "affiliation-city": "St Andrews",
            "affilname": "University of St Andrews",
            "affiliation-country": "United Kingdom"
        },
        {
            "affiliation-city": "Berlin",
            "affilname": "Technical University of Berlin",
            "affiliation-country": "Germany"
        },
        {
            "affiliation-city": "Bangalore",
            "affilname": "Samsung Research Institu

In [11]:
df_test = pd.DataFrame.from_dict(entry['affiliation'], orient='columns')
df_test

,affiliation-city,affilname,affiliation-country
0,Lahore,Information Technology University,Pakistan
1,Trieste,Abdus Salam International Centre for Theoretic...,Italy
2,Cambridge,University of Cambridge,United Kingdom
3,St Andrews,University of St Andrews,United Kingdom
4,Berlin,Technical University of Berlin,Germany
5,Bangalore,Samsung Research Institute,India


In [12]:
df_extra_info_test = pd.DataFrame(dict_extra_info)
df_extra_info_test

,0,1
affiliation,"[{'affiliation-city': 'Lahore', 'affilname': '...","[{'affiliation-city': 'Lahore', 'affilname': '..."
coredata,"{'srctype': 'j', 'eid': '2-s2.0-84979266854', ...","{'srctype': 'j', 'eid': '2-s2.0-84979266854', ..."


In [13]:
df_extra_info_test_transposed = df_extra_info_test.T
df_extra_info_test_transposed

,affiliation,coredata
0,"[{'affiliation-city': 'Lahore', 'affilname': '...","{'srctype': 'j', 'eid': '2-s2.0-84979266854', ..."
1,"[{'affiliation-city': 'Lahore', 'affilname': '...","{'srctype': 'j', 'eid': '2-s2.0-84979266854', ..."


In [14]:
df_combined = pd.concat([df_current_extra_info, df_extra_info_test_transposed],ignore_index=True)
df_combined

,affiliation,coredata
0,"[{'affiliation-city': None, 'affilname': 'Pata...","{'srctype': 'j', 'prism:issueIdentifier': '230..."
1,"[{'affiliation-city': 'New Delhi', 'affilname'...","{'srctype': 'j', 'eid': '2-s2.0-85083171050', ..."
2,None,None
3,"[{'affiliation-city': 'London', 'affilname': '...","{'srctype': 'j', 'eid': '2-s2.0-79953057246', ..."
4,"[{'affiliation-city': 'Baoding', 'affilname': ...","{'srctype': 'j', 'prism:issueIdentifier': '7',..."
...,...,...
11661,"{'affiliation-city': 'Safat', 'affilname': 'Al...","{'srctype': 'j', 'prism:issueIdentifier': '1',..."
11662,"[{'affiliation-city': 'Atlanta', 'affilname': ...","{'srctype': 'j', 'prism:issueIdentifier': '1',..."
11663,"[{'affiliation-city': 'Rio de Janeiro', 'affil...","{'srctype': 'j', 'eid': '2-s2.0-85083879171', ..."
11664,"[{'affiliation-city': 'Lahore', 'affilname': '...","{'srctype': 'j', 'eid': '2-s2.0-84979266854', ..."
